In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import time


In [2]:
import os
import pymongo
import re
import pickle
import json
import torch
import time
import pandas as pd
from tqdm import tqdm

from langchain_community.document_loaders.merge import MergedDataLoader

from langchain.docstore.document import Document
from langchain.chains.query_constructor.schema import AttributeInfo
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.retrievers import BM25Retriever, EnsembleRetriever


from langchain.embeddings import HuggingFaceEmbeddings
from transformers import AutoModel, AutoTokenizer
from langchain.vectorstores import Chroma
import json
from bson import ObjectId

from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from llama_index.llms import HuggingFaceLLM
from llama_index.prompts.prompts import SimpleInputPrompt
from langchain.chains import LLMChain

/home/synthesisproject/anaconda3/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
# To load the sentencevectordb

embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
persist_directory = 'data/chroma/sminilm'

svdb = Chroma(persist_directory=persist_directory, embedding_function=embeddings)
print(svdb._collection.count())

4541573


In [7]:
def clear_cache():
  if torch.cuda.is_available():
    model = None
    torch.cuda.empty_cache()

In [8]:
model = None
llm = None
clear_cache()
tokenizer = None

In [5]:
from langchain import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline
import torch 
MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.2"
 
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
 
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, torch_dtype=torch.float16, trust_remote_code=True, device_map="auto"
)
 
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
generation_config.max_new_tokens = 200
generation_config.temperature = 0.0001
generation_config.top_p = 0.90
generation_config.do_sample = True
generation_config.repetition_penalty = 1.15
 
text_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    generation_config=generation_config,
)
 
llm = HuggingFacePipeline(pipeline=text_pipeline, model_kwargs={"device":'cuda'})



Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [10]:
llm("The first man on the moon was ...")

/home/synthesisproject/anaconda3/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


" Neil Armstrong.\n\nThe second man on the moon was... Buzz Aldrin.\n\nThe third man on the moon was... Michael Collins.\n\nMichael Collins? Who's that, you ask? Well, he was the guy who stayed in the spacecraft while Armstrong and Aldrin went down to the lunar surface. He piloted the Command Module Columbia as it orbited above the Moon, keeping an eye out for any potential problems with the Lunar Module Eagle. When Armstrong and Aldrin returned from their historic moonwalk, they re-docked with the Columbia, and all three astronauts made it safely back to Earth.\n\nSo why don't we hear much about Michael Collins these days? It might be because his role in the mission was largely behind the scenes, but there are other reasons too. For one thing, he didn't get to set foot on the Moon like his two colleagues did. And"

In [11]:

material_name='Q1'


search=f'What are the characteristics, origin and abbreviation of the material named {material_name}?'
table_caption='Chemical composition of major elements, expressed in wt% of oxides, for stone and quicklime materials. Loss on Ignition (LOI).'
doi="10.1016/j.conbuildmat.2017.06.147"

# Load the QA chain
chain = load_qa_chain(llm, chain_type="stuff")


# Document retrieval (adjust as needed for your database)
try: 
    docs = svdb.similarity_search(search, k=50, filter={"doi":doi })

except Exception as e:
    docs = svdb.similarity_search(search, k=20, filter={"doi":doi })


tab = tvdb.similarity_search('', k=1, filter={"doi":doi })

# Document retrieval (adjust as needed for your database)

abstract = avdb.similarity_search("", k=1, filter={"doi":doi })

found_docs =[]

for doc in docs:
    if f'({material_name})' in doc.page_content.split():
        found_docs.append(doc)
    elif f'({material_name}' in doc.page_content.split():
        found_docs.append(doc)
    elif f'{material_name})' in doc.page_content.split():
        found_docs.append(doc)
        
if len(found_docs)<5:
    for doc in docs:
        if material_name in doc.page_content.split():
            found_docs.append(doc)           


if len(found_docs)>10: 
    found_docs = found_docs[:10]

if len(found_docs)==0: 
    found_docs = docs[:10]


# Task-specific prompt

if len(abstract)==1:
    prompt_template = (
        f"""[INST] <<SYS>>\
        You are a helpful, respectful, and honest assistant. Always strive to provide the most helpful and accurate answers. 
        If a question is unclear, nonsensical, or lacks factual coherence, clarify or explain the issue instead of giving an incorrect response.         If no relevant information about the material is available, respond with NaN.\n\n

        <</SYS>>\n\n
        Task: Describe in one sentence the composition or characteristics of the material called {material_name}, using only the provided information. 
        Related Table:\n\n {table_caption}\n{tab[0].metadata['table']} \n\n
        Abstract: {abstract[0].page_content}
        [/INST]"""

    )
else: 
    prompt_template = (
        f"""[INST] <<SYS>>\
        You are a helpful, respectful, and honest assistant. Always strive to provide the most helpful and accurate answers. 
        If a question is unclear, nonsensical, or lacks factual coherence, clarify or explain the issue instead of giving an incorrect response.         If no relevant information about the material is available, respond with NaN.\n\n

        <</SYS>>\n\n
        Task: Describe in characteristics of the material called {material_name}, using only the provided information. One sentence only.
        Material name: {material_name}\n\n
        Related Table:\n\n {table_caption}\n{tab[0].metadata['table']} \n\n
        [/INST]"""

    )
# Create a PromptTemplate object
prompt_template = PromptTemplate(
    template=prompt_template,
    input_variables=["user_question"]
)


chain = LLMChain(llm=llm, prompt=prompt_template,metadata=None)



# Run the chain with the retrieved documents and custom prompt
res = chain.run(input_documents=found_docs, question=material_name)
print(res)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 The described material Q1 refers to quicklime, specifically two types derived from the same rock but produced through traditional and industrial kilns. Its chemical composition includes calcium oxide (CaO) and calcium hydroxide (Ca(OH)2), with varying surface free energies and electrokinetic properties depending on the production method. Traditional quicklime exhibits greater hydrophilicity and higher ξ Potential compared to industrial quicklime due to differences in calcination temperatures and charge introduction during manufacturing.


In [12]:
docs

[]

In [13]:

# Load the file
materials_df  = pd.read_csv('Final_ext_compositions.csv',encoding='cp1252')

# Display the first few rows of the new DataFrame
materials_df.head()


,Material name,SiO2,Al2O3,Fe2O3,CaO,MgO,SO3,TiO2,MnO,K2O,Na2O,LOI,Caption,DOI
6761,GGBFS +,34.5,12.70,0.50,41.3,5.90,NaN,NaN,NaN,NaN,NaN,0.30,Chemical compositions and physical properties ...,10.1016/j.conbuildmat.2018.01.095
6762,Cement,19.61,7.32,3.32,63.15,2.54,2.13,NaN,NaN,NaN,NaN,2.97,Chemical compositions of cement and FBA (%).,10.1016/j.jclepro.2015.03.007
6763,FBA,52.1,18.34,11.99,6.61,4.85,NaN,0.87,NaN,1.57,2.43,4.13,Chemical compositions of cement and FBA (%).,10.1016/j.jclepro.2015.03.007
6764,Cement,20.1,5.30,3.70,63.6,1.20,1.6,NaN,NaN,0.30,0.50,2.20,Physical and chemical properties of cementitio...,10.1016/j.jclepro.2017.04.105
6765,Slag,35.9,7.90,0.50,38.4,8.10,0.3,NaN,NaN,0.90,0.50,2.40,Physical and chemical properties of cementitio...,10.1016/j.jclepro.2017.04.105


In [73]:
def save_responses(responses):
    # Implement the logic to save 'responses'
    # For example, saving to a file:
    filename = f'descriptions.csv'
    # Convert responses to DataFrame and save as CSV
    pd.DataFrame(responses, columns=['Material Name','Response', 'Caption', 'DOI']).to_csv(filename, index=False)
    
# Placeholder for responses
responses = []
m_is=[]

In [ ]:

for i, row in enumerate(tqdm(materials_df.iterrows(), total=materials_df.shape[0])):
    try:

        material_name = row[1]['Material name']
        doi = row[1]['DOI']
        caption = row[1]['Caption']

        search=f'Define what is this material: {str(material_name)}?'
        table_caption=caption

        # Load the QA chain
        chain = load_qa_chain(llm, chain_type="stuff")

        # Document retrieval (adjust as needed for your database)

        docs = svdb.similarity_search(search, k=20, filter={"doi":str(doi) })

        tab = tvdb.similarity_search(caption, k=1, filter={"doi":str(doi) })

        # Document retrieval (adjust as needed for your database)

        abstract = avdb.similarity_search(search, k=5, filter={"doi":str(doi) })

        found_docs =[]


        for doc in docs:
            if f'({material_name})' in doc.page_content:
                found_docs.append(doc)
            elif f'({material_name}' in doc.page_content:
                found_docs.append(doc)
            elif f'{material_name})' in doc.page_content:
                found_docs.append(doc)

        if len(found_docs)<5:
            for doc in docs:
                if material_name in doc.page_content:
                    found_docs.append(doc)           


        if len(found_docs)>10: 
            found_docs = found_docs[:10]

        if len(found_docs)==0: 
            found_docs = docs[:10]


        exp = ''

        for doc in found_docs:
            exp+=doc.page_content

        abss = ''

        for doc in abstract:
            abss+=doc.page_content    
            
            
        prompt_template = (
            f"""[INST] <<SYS>>\
            As a helpful, respectful, and honest assistant, your primary goal is to provide the most accurate and helpful answers possible. In cases where a question is unclear, nonsensical, or lacks factual coherence, prioritize clarification or explanation over providing an incorrect response. 
            <</SYS>>\n\n
            Describe the material named '{material_name}' in one sentence only. If information about the material is unavailable, respond with 'Not Available (NaN)' and avoid assumptions or discussions about other materials and avoid talking about the chemical composition of the material. Focus on the following details:

            Material Name: {material_name}\n\n
            Related Table: {table_caption}{tab[0].metadata['table']}:\n\n
            {abss} \n
            {exp}
            [/INST]"""

            )
        # Create a PromptTemplate object
        prompt_template = PromptTemplate(
            template=prompt_template,
            input_variables=["user_question"]
        )


        chain = LLMChain(llm=llm, prompt=prompt_template,metadata=None)

        if len(abstract)>=1:
            abstract.append(found_docs)
            found_docs = abstract
        


        # Run the chain with the retrieved documents and custom prompt
        res = chain.run(input_documents=found_docs, question=material_name)
        responses.append((material_name, res,caption,doi))
    except Exception as e:
        m_is.append(i)
    # Check if it's time to save the responses
    if (i + 1) % 50 == 0:
        save_responses(responses)
        
        